### Coding GPT from scratch:
(Only using pyTorch for neural network training and encoding)

This projects layouts the base structure and architecture for a language model, skipping the training and finetuning on large data (for obvious purposes). This is a character based LLM, rather than a whole word one, for simplicity purposes.

Its main purpose is to learn and consolidate my knowledge of the transformer architecture. I plan to follow it up, building the neural network from scratch (which I have already done in C), and then explore more advanced topics of LLMs, such as RAG, CAG, LPMs, etc.

In [1]:
import requests
import numpy as np

In [2]:
# Extracts the Tiny-Shakespeare and stores it in a variable:
url ="https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
r = requests.get(url)
if r.status_code == 200:
    text = r.text
    print("Dataset extracted. \n")
else:
    print(f"Failed to fetch. Status: {r.status_code}")

print(text[:200])


Dataset extracted. 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [3]:
#Organizes the vocabulary (in this case unique characters, rather than words)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(" \n Vocabulary size: "+ str(vocab_size))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
 
 Vocabulary size: 65


In [4]:
#Tokenizing the input text, encoding individual characters into vectors or integers
string_to_token = { ch: i for i, ch in enumerate(chars)} #creates dictionary with character as key and token as value
token_to_string = {i: ch for i, ch in enumerate(chars)} #creates dictionary with token as key and character as value

encode = lambda s : [string_to_token[c] for c in s] #given a string, give the corresponding series of tokens
decode = lambda l : ''.join([token_to_string[i] for i in l])#given a series of tokens, return the strings (concats into a single string)

print(encode("Hobbits"))
print(decode(encode("Hobbits")))
#Google uses Sentrencepeice for encoding, which encodes subwords.
#OpenAI uses tiktoken, which is a BPE tokenizer, with around 50k tokens.
#There is a trade-off between vocabulary size and token size. Thus subword is often used.

[20, 53, 40, 40, 47, 58, 57]
Hobbits


In [5]:
#Tokenizing the whole tinyshakespeare dataset, using pyTorch:
import torch
data = torch.tensor(encode(text), dtype = torch.long) # the data tensor is tikenized in its entirety
data.shape, data.dtype

(torch.Size([1115394]), torch.int64)

In [6]:
print(data[:200]) #tokenized the full dataset into a sequence of integers

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [7]:
#splitting into training and validation sets, 90% for training
n = int(0.9*len(data)) #casts 0.9*len(data) to int
train_data = data[:n] #indexes 90% as training
val_data = data[n:] #the rest is validation, checking for overfitting

In [8]:
#feeding the text into the transformer by chunking the dataset and then sample it for training
block_size = 8 #AKA context window size
train_data[:block_size+1] #first context window of the dataset

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
#the training is done by guessing the next word, but for every single subsequence as well, in parallel
#the chunks of 9 characters actually only contain 8 examples, since the first word is the seed and must be present.
x = train_data[:block_size] #1st window, context
y = train_data[1:block_size+1] #2nd sliding window, it is the target
for t in range(block_size):
    context= x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is: {target}")

#The training is done from context 1 to block_size in parallel. It makes it more efficient to train. 
#And allows the transformer to percieve long and short term relationships between words, form 1 to block_size 

When input is tensor([18]), the target is: 47
When input is tensor([18, 47]), the target is: 56
When input is tensor([18, 47, 56]), the target is: 57
When input is tensor([18, 47, 56, 57]), the target is: 58
When input is tensor([18, 47, 56, 57, 58]), the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [10]:
#multiple blocks are computed in parallel into the transformer model, taking advantage of the GPUs.
#this batching is done for efficiency, and they are processed independently

torch.manual_seed(1337) #makes RNG stable for control and debugging purposes
batch_size = 4
block_size = 8

def get_batch(split):
    #generates the batch with inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #generates batch_size random positions between 0 and the total indexable training set
    x = torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb)

print('Targets:')
print(yb.shape)
print(yb)
print("==============")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()}, the target is: {target}")

Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When input is [24], the target is: 43
When input is [24, 43], the target is: 58
When input is [24, 43, 58], the target is: 5
When input is [24, 43, 58, 5], the target is: 57
When input is [24, 43, 58, 5, 57], the target is: 1
When input is [24, 43, 58, 5, 57, 1], the target is: 46
When input is [24, 43, 58, 5, 57, 1, 46], the target is: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43], the target is: 39
When input is [44], the target is: 53
When input is [44, 53], the target is: 56
When input is [44, 53, 56], the target is: 1
When input is [44, 53, 56, 1], the target is: 58
When input is [44, 53, 56, 

In [11]:
#feeding it into a simple language model, the Bigram Language Model:

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) #makes RNG stable for control and debugging purposes

class BigramLanguageModel(nn.Module): #subclass of nn module

    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # creates the embedding table
    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx) #takes the input and looks it up in the embedding table. 
        # 24 takes the 24th row from the embedding table
        # PyTorch arranges it into a Batch, Time, Channel tensor: (B=batch_size=4, T=block_size=8, C=vocab_size=65)
        if targets is None:
            loss =None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) #reshapes logit vector
            targets = targets.view(B*T) #reshapes target vector
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get predictions
            logits, loss = self(idx)
            #focus only on the last time step
            logits = logits[:,-1,:] #(B,C)
            #apply softmax
            probs = F.softmax(logits, dim =-1) # (B,C)
            # sample from distribution randomly, generating prediction for each batch
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            #append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss) #Since we have 65 distinct vocab items, the expected loss is -log(1/65) = 4.174

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [12]:
 #creates a seed token as 0
print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens=100)[0].tolist())) #creates 100 character/token generation, from the 0th batch
#it makes no sense, as it hasn't been trained 


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [13]:
#training the bigram model:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
torch.cuda.get_device_name(0) #checking if my GPU is being used

'NVIDIA GeForce RTX 4070'

In [15]:
batch_size = 32

for steps in range (10000):
    #sample new batch of data
    xb,yb = get_batch('train')

    #calculate loss
    logits, loss = m(xb,yb) #evaluate the loss
    optimizer.zero_grad(set_to_none=True) #zeroing 
    loss.backward() #getting the gradients through backprop
    optimizer.step() #updates parameters

print(loss.item()) #loss is improving

2.5727508068084717


In [16]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


### Creating self-attention
Self-attention allows for embeddings to change meaning/share information based on context, potentially over a large distance in the text.

In [17]:
#mathematical trick, toy example
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

The 8 tokens in the batch are uncoupled, and do not communicate. Tokens should not communicate with future tokens.

So tokens at $T_i$ should only interact with tokens $T_{i-1}$ and so on. Information flows from previous context only.

One (weak) way of doing this is through the average of preceding tokens. It loses spatial information, for instance.

In [18]:
#for every token, calculate the average of all previous tokens (itself included)
#x[b,t] = mean_{i<=t} x [b,i]
xbow = torch.zeros((B,T,C))#BAG OF WORDS
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev,0) #averages time


In [19]:
x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [20]:
a = torch.tril(torch.ones(3,3)) #returns the lower triangular portion of the ones matrix
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [21]:
#Matrix multiplication allows for optimization
torch.manual_seed(42)
b = torch.randint(0,10,(3,2)).float()
c = a @ b 
print('a= \n')
print(a)
print('b= \n')
print(b)
print('c= \n')
print(c)

a= 

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b= 

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= 

tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [22]:
#this allows cumulative sums of the lines in matrix a in steps
#we are essentialy doing the sum of a variable number of rows in a. 
#average can be done in incremental fashion by normalizing each row
a = torch.tril(torch.ones(3,3))
a= a/torch.sum(a,dim=1, keepdim=True)
c = a @ b 
print('a= \n')
print(a)
print('b= \n')
print(b)
print('c= \n')
print(c)

a= 

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b= 

tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= 

tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [23]:
#making the averaging code more efficient:
wei = torch.tril(torch.ones(T,T)) #weights
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [24]:
xbow2 = wei @ x #(BxTxT) @ (BxTxC) --> (BxTxC)
torch.allclose(xbow,xbow2)

False

In [25]:
xbow[1],xbow2[1]

(tensor([[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]]),
 tensor([[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]]))

In [26]:
#v3: softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) #tokens from the future  cannot communicate
wei = F.softmax(wei, dim = -1) #goes through softmax to generate a pdf
xbow3 = wei @ x #this is the step that actually updates the embeddings (aggregation)
torch.allclose(xbow2, xbow3)

True

In [27]:
wei #softmax of the masked dist with zeroes as -inf

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

How much of each token from the past must we incorporate? It depends on the wei matrix, of course.

But different words have different interactions between themselves. Tokens cannot communicate to tokens from the future.

So essentially, weighted aggregations from past context/elements can be done with a matrix multiplication with lower triangular fashion. The elements in the lower part communicate how much meaning or interaction they must communicate or fuse.

In [45]:
# self-attention:
torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.randn(B,T,C)

#single head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C,head_size, bias=False)
value = nn.Linear(C,head_size, bias=False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
#communication:

wei = q @ k.transpose(-2,-1) # (B, T, 16) @ (B, 16, T) == (B,T,T)
tril= torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei,dim=-1)

v = value(x)
out = wei @ v


out.shape

torch.Size([4, 8, 16])

In [46]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [47]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

These weights obviously change between letters/tokens -> gather information through the past in a data-depend weight

Every single token in the array will emit a query and a key vector. Query: what I'm looking for. Key: what I contain.
The attention pattern times the values matrix is the new weight matrix, constructed by the dot product between all keys and queries.

Attention can be applied to any directed graph. There is no notion to space, this is why the tokens are positionally encoded.

The batches do not communicate.

The future tokens do not communicate to past tokens in LLMs. Sentiment analysis is one example of fully connected tokens (because it is not predictive), so a self-attention ENCODER block is used.

For LLMs, the previously described construction is a DECODER block, with the -inf mask, so that the future values are not available to the past.

Self-attention has keys, queries and values coming from the same source, x. Cross-attention takes key, queries and values from different sources.

$Attention(Q,K,V) = softmax(\frac{Q\cdot K^T}{\sqrt{d_k}}) \cdot V$

Attention is scaled by $\sqrt{d_k}$ because the variance of the weight matrix would be in the order of the head size $d_k$.
If attention is normalized, the variance will be 1. Wei will be fed into softmax, so it must be fairly diffuse to not saturate (they will converge to one-hot vectors otherwise).

In [60]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) 
k.var(), q.var(), wei.var()

(tensor(0.9400), tensor(1.0119), tensor(15.1440))

In [61]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5
k.var(), q.var(), wei.var()

(tensor(1.0172), tensor(0.9599), tensor(0.8116))

In [62]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [64]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*10,dim=-1) #sharpens the pdf towards the max

tensor([1.5851e-02, 7.8918e-04, 1.1713e-01, 7.8918e-04, 8.6545e-01])

In [ ]:
#head of attention module
n_embd =8
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        #compute attention
        wei = q @ k.transpose(-2,-1) *C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
        wei = F.softmax(wei, dim =-1)
        v= self.value(x)
        out = wei @ v
        return out
    
# this is to be instantiated before the lm head

In [ ]:
# implementing multi-head attention: applying multiple attention layers in parallel and sum results
